In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import yfinance as yf

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

import os
import random

# -----------------------------
# Set seeds for reproducibility
# -----------------------------
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# -----------------------------
# Enable GPU memory growth
# -----------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU enabled and memory growth set.")
    except RuntimeError as e:
        print(e)

# -----------------------------
# Constants
# -----------------------------
TICKER = "AAPL"
LOOKBACK = 60
BATCH_SIZE = 32
EPOCHS = 20

print("Setup complete. Ready to download data and train LSTM models.")


/opt/anaconda3/envs/tf_env_clean/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Setup complete. Ready to download data and train LSTM models.


In [ ]:
# Download 20 years of daily closing prices
data = yf.download(TICKER, start="2005-01-01", end="2025-01-01")
print(f"Downloaded {len(data)} rows for {TICKER}.")

# Drop missing values
data = data.dropna()
print(f"After dropping NA: {len(data)} rows remaining.")

# Use only the closing price
close_prices = data['Close'].values.reshape(-1, 1)

# Basic statistics
print("Closing price stats:")
print(f"Min: {close_prices.min():.2f}")
print(f"Max: {close_prices.max():.2f}")
print(f"Mean: {close_prices.mean():.2f}")
print(f"Std: {close_prices.std():.2f}")

# Preview the first 10 rows
print(data.head(10))

# Save for future use
data.to_csv(f"{TICKER}_historical.csv")
print(f"Saved historical data to {TICKER}_historical.csv")


In [ ]:
plt.figure(figsize=(14,6))
plt.plot(data['Close'], label='Closing Price', color='blue')
plt.title(f"{TICKER} Closing Price Over Time")
plt.xlabel("Date")
plt.ylabel("Price USD")
plt.legend()
plt.grid(True)
plt.show()

# Plot moving averages
data['MA50'] = data['Close'].rolling(window=50).mean()
data['MA200'] = data['Close'].rolling(window=200).mean()

plt.figure(figsize=(14,6))
plt.plot(data['Close'], label='Close', alpha=0.5)
plt.plot(data['MA50'], label='50-day MA', color='orange')
plt.plot(data['MA200'], label='200-day MA', color='red')
plt.title(f"{TICKER} Moving Averages")
plt.xlabel("Date")
plt.ylabel("Price USD")
plt.legend()
plt.show()

# Plot histogram of returns
data['Returns'] = data['Close'].pct_change()
plt.figure(figsize=(10,4))
plt.hist(data['Returns'].dropna(), bins=100, alpha=0.7, color='green')
plt.title(f"{TICKER} Daily Returns Distribution")
plt.xlabel("Return")
plt.ylabel("Frequency")
plt.show()
